<a href="https://colab.research.google.com/github/aliseyfi75/BP-Estimation-PPG/blob/main/Codes/CNN_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
from tensorflow import keras

In [3]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout
from keras.optimizers import SGD
from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import EarlyStopping

In [4]:
xtest = np.fromfile('/content/drive/MyDrive/CPSC 533 project/xtest.dat', dtype= complex)
xtrain = np.fromfile('/content/drive/MyDrive/CPSC 533 project/xtrain.dat', dtype= complex)
ytest = np.fromfile('/content/drive/MyDrive/CPSC 533 project/ytest.dat', dtype= int)
ytrain = np.fromfile('/content/drive/MyDrive/CPSC 533 project/ytrain.dat', dtype = int)

In [59]:
def CNN_model():
  model = Sequential()
  model.add(Conv2D(64, kernel_size=3, activation='tanh', input_shape=(501,244,1)))
  model.add(MaxPooling2D())
  model.add(Conv2D(64, kernel_size=3, activation='tanh'))
  model.add(MaxPooling2D())
  model.add(Conv2D(64, kernel_size=3, activation='tanh'))
  model.add(Conv2D(64, kernel_size=3, activation='tanh'))
  model.add(Conv2D(64, kernel_size=3, activation='tanh'))
  model.add(MaxPooling2D())
  model.add(Flatten())
  model.add(Dense(10, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(2, activation='relu'))
  model.compile(loss='mean_absolute_error', optimizer='adam')  

  return model

In [7]:
xtrain = xtrain.reshape((-1,501,244,1)).real
xtest = xtest.reshape((-1,501,244,1)).real
ytrain = ytrain.reshape((-1,2))
ytest = ytest.reshape((-1,2))

print(xtest.shape, ytest.shape)

(23, 501, 244, 1) (23, 2)


In [60]:
with tf.device('/device:GPU:0'):
  # Fit CNN model
  model = CNN_model()
  es = EarlyStopping(monitor='val_loss', mode='auto', verbose=1, patience=100)

  model.fit(xtrain, ytrain, batch_size=256,epochs=400, validation_split=0.2, verbose=1, callbacks=[es])

  model.summary()

Epoch 1/400
1/1 [==============================] - 0s 161ms/step - loss: 102.6039 - val_loss: 105.2484
Epoch 2/400
1/1 [==============================] - 0s 46ms/step - loss: 100.8738 - val_loss: 97.7946
Epoch 3/400
1/1 [==============================] - 0s 47ms/step - loss: 94.0647 - val_loss: 80.1782
Epoch 4/400
1/1 [==============================] - 0s 47ms/step - loss: 78.0283 - val_loss: 52.5892
Epoch 5/400
1/1 [==============================] - 0s 47ms/step - loss: 54.1871 - val_loss: 47.3691
Epoch 6/400
1/1 [==============================] - 0s 46ms/step - loss: 62.4747 - val_loss: 35.4100
Epoch 7/400
1/1 [==============================] - 0s 47ms/step - loss: 46.7161 - val_loss: 32.9029
Epoch 8/400
1/1 [==============================] - 0s 48ms/step - loss: 35.5136 - val_loss: 42.5395
Epoch 9/400
1/1 [==============================] - 0s 47ms/step - loss: 45.3428 - val_loss: 40.0071
Epoch 10/400
1/1 [==============================] - 0s 52ms/step - loss: 41.8637 - val_loss: 26.

In [61]:
# Find the prediction and error
ypred = model.predict(xtest)

print(ypred, ytest)

errors = abs(ytest - ypred)

print('Mean Absolute Error:', np.mean(errors, axis=0))
print('STD of error:', np.std(errors, axis=0))

[[105.757164  61.774963]
 [107.20004   62.436287]
 [ 93.60318   54.719467]
 [147.45012   76.84673 ]
 [122.43873   66.41627 ]
 [117.192276  62.489273]
 [119.45618   65.58544 ]
 [119.45618   65.58544 ]
 [132.42686   71.42524 ]
 [120.24337   65.46341 ]
 [129.80951   69.758255]
 [131.74037   70.65908 ]
 [136.5381    74.72337 ]
 [134.71272   72.9585  ]
 [119.45618   65.58544 ]
 [125.78186   65.30934 ]
 [117.456375  64.60461 ]
 [129.86967   69.447464]
 [114.970825  63.93948 ]
 [129.35118   70.42329 ]
 [137.44933   71.467545]
 [107.411865  63.67698 ]
 [ 98.476814  54.304382]] [[137  61]
 [107  63]
 [  0   0]
 [147  69]
 [143  77]
 [138  65]
 [112  63]
 [124  75]
 [114  53]
 [166  75]
 [132  61]
 [123  66]
 [153  67]
 [131  66]
 [135  76]
 [197  93]
 [132  77]
 [125  70]
 [117  77]
 [149  79]
 [146  78]
 [120  66]
 [115  78]]
Mean Absolute Error: [19.116061   10.88268927]
STD of error: [17.824281   11.05354714]
